In [52]:
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd

from collections import defaultdict
from tqdm import tqdm

import numpy as np

In [3]:
model = KeyedVectors.load_word2vec_format("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Recipe1M/vocab.bin/vocab.bin", binary=True)

In [20]:
df=pd.read_csv("F:/SHRINIVAS/KGP/BTP/Meal-Planning/rec.csv")

In [21]:
df.head()

,Unnamed: 0,recipe_id,recipe_name,review_nums,category,aver_rate,image_url,ingredients,cooking_directions,nutritions,reviews,tags
0,0,6698,Mom's Zucchini Bread,9355,appetizer,4.754207,https://images.media-allrecipes.com/userphotos...,all-purpose flour^salt^baking soda^baking powd...,{'directions': u'Prep\n20 m\nCook\n1 h\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{9568256: {'rating': 5, 'followersCount': 0, '...",north-american;breads;easy;beginner-cook;inexp...
1,1,6709,Zucchini Walnut Bread,58,appetizer,4.478873,http://images.media-allrecipes.com/userphotos/...,chopped walnuts^eggs^white sugar^vegetable oil...,"{'directions': u'Whisk together flour, baking ...","{u'niacin': {u'hasCompleteData': True, u'name'...","{1636096: {'rating': 4, 'followersCount': 0, '...",weeknight;breads;fruit;vegetables;kid-friendly...
2,2,6725,Honey Wheat Bread I,2054,appetizer,4.766846,http://images.media-allrecipes.com/userphotos/...,rapid rise yeast^white sugar^warm water (110 d...,{'directions': u'Prep\n25 m\nCook\n35 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1542146: {'rating': 4, 'followersCount': 56, ...",healthy;breads;dietary;low-cholesterol;healthy...
3,3,6730,Chocolate Chip Orange Zucchini Bread,503,dessert,4.674833,https://images.media-allrecipes.com/userphotos...,eggs^white sugar^vegetable oil^vanilla extract...,"{'directions': u'Sift together flour, baking p...","{u'niacin': {u'hasCompleteData': True, u'name'...","{1978376: {'rating': 5, 'followersCount': 0, '...",for-large-groups;healthy;breads;kid-friendly;d...
4,4,6732,Chocolate Chip Pumpkin Bread,866,dessert,4.718112,https://images.media-allrecipes.com/userphotos...,white sugar^pumpkin puree^vegetable oil^water^...,{'directions': u'Prep\n30 m\nCook\n1 h\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1830917: {'rating': 3, 'followersCount': 0, '...",breads;fall;winter;seasonal;quick-breads;4-hou...


In [54]:
ings=df['ingredients'].values
titles=df['recipe_name'].values

In [23]:
vocab=set(model.key_to_index.keys())

In [26]:
freq=defaultdict(int)
for ing in ings:
    for val in ing.split('^'):
        for v in val.split():
            freq[v]+=1

In [41]:
maxV=max(freq.items(),key=lambda x: x[1])
for k,v in freq.items():
    freq[k]=v/maxV[1]

In [42]:
DIM=300

In [56]:
title2vec={}
vecs=[]
for id,ing in enumerate(tqdm(ings)):
    vec=[0]*DIM
    for val in ing.split('^'):
        for v in val.split():
            v=v.replace('-','_')
            if v in vocab:
                vec+=model.get_vector(v,norm=True)*freq[v]
    title2vec[titles[id]]=vec
    vecs.append(vec)

100%|██████████| 7280/7280 [00:02<00:00, 3030.47it/s]


In [57]:
id=10
titles[10]

'Amish White Bread'

In [58]:
def sim(v1,v2):
    return np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

Might need some improvement as chicken is coming close to patato soup for some reason at id=1000, 1021

In [71]:
i=2132
v1=vecs[i]
scores=[]
for id,v in enumerate(vecs):
    scores.append([sim(v1,v),id])

scores=sorted(scores,reverse=True)

for rec in scores[:5]:
    print(titles[rec[1]])


Creole Gumbo
Chicken Pot Pies with Puff Pastry
Cream of Brie Soup
Oven Baked Jambalaya
Chicken, Rice and Vegetable Soup


In [65]:
vecs_np=np.array(vecs)

In [67]:
np.save("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Processed/rec_vecs.npy",vecs_np)